In [1]:
from dotenv import load_dotenv

load_dotenv('../../.env')

True

# Upload Files to KB

In [ ]:
from openai import OpenAI

client = OpenAI()
file = client.beta.files.upload(
	file=open("tech_manual.pdf", "rb"),
	filetype="application/pdf",
	description="Tech product manual"
)

In [ ]:
assistant = client.beta.assistants.create(
  instructions="You are a tech support chatbot. Use the product manual to respond accurately to customer inquiries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [ ]:
thread = client.beta.threads.create()
   message = client.beta.threads.messages.create(
       thread_id=thread.id,
       role="user",
       content="How do I reset my Model X device?",
   )

# Run Thread

In [ ]:
run = client.beta.threads.runs.create(
   thread_id=thread.id,
   assistant_id=assistant.id,
)

# the run will enter the queued state before it continues it’s execution.

# Information Retrieval

In [ ]:
messages = client.beta.threads.messages.list(
	thread_id=thread.id
)

assistant_response = messages.data[0].content[0].text.value

# Hugging Face

In [4]:
import os
from huggingface_hub import HfApi

hf_api = HfApi(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

# Run Inference

In [5]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [6]:
import requests
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {HUGGINGFACEHUB_API_TOKEN}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query("Can you please let us know more")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Tasks

In [ ]:
import requests

API_TOKEN = 'your_api_token_here'
model_name = 'facebook/bart-large-cnn'

text_to_summarize = "Hugging Face's API simplifies accessing powerful NLP models for tasks like summarization, transforming verbose texts into concise, insightful summaries."

endpoint = f'https://api-inference.huggingface.co/models/{model_name}'
headers = {'Authorization': f'Bearer {API_TOKEN}'}
data = {'inputs': text_to_summarize}

response = requests.post(endpoint, headers=headers, json=data)
summarized_text = response.json()[0]['summary_text']

print(summarized_text)

# Verifiy Model Tasks

In [ ]:
import requests

headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

data = query({"inputs": "I love how this app simplifies complex tasks effortlessly . I'm frustrated by the frequent errors in the software's latest update"})
print(data)

# Stable Diffusion pipelines

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "Create an image of a futuristic cityscape on an alien planet, featuring towering skyscrapers with glowing neon lights, a sky filled with multiple moons, and inhabitants of various alien species walking through vibrant market streets"
image = pipe(prompt).images[0]

image.save("astronaut_rides_horse.png")

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["GAIA's questions are rooted in practical use cases, requiring AI systems to interact with a diverse and uncertain world, reflecting real-world applications.", " GAIA questions require accurate execution of complex sequences of actions, akin to the Proof of Work concept, where the solution is simple to verify but challenging to generate."]

model = SentenceTransformer('Equall/english-beta-0.3', use_auth_token=API_TOKEN)
embeddings = model.encode(sentences)
print(embeddings)

# Image Captioning

In [ ]:
from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

image_to_text("https://ankur3107.github.io/assets/images/image-captioning-example.png")

# [{'generated_text': 'a soccer game with a player jumping to catch the ball '}]

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [7]:
import requests

# Function to fetch text from the API
def fetch_text_from_api():
    url = "https://lexper.p.rapidapi.com/v1.1/extract"
    querystring = {
        "url": "https://techcrunch.com/2023/11/25/neuralink-elon-musks-brain-implant-startup-quietly-raises-an-additional-43m/",
        "js_timeout": "30",
        "media": "true"
    }
    headers = {
        "X-RapidAPI-Key": "xxx",
        "X-RapidAPI-Host": "lexper.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    # Extract the relevant text from the API response
    # Adjust the following line according to the structure of your API response
    return data.get('article', {}).get('text', '')

# Function to summarize the text using Hugging Face API
def query_huggingface(payload):
    API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    headers = {"Authorization": f"Bearer {HUGGINGFACEHUB_API_TOKEN}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Fetch the text
text_to_summarize = fetch_text_from_api()

# Summarize the text
summarization_payload = {
    "inputs": text_to_summarize,
    "parameters": {"do_sample": False},
}

summary_response = query_huggingface(summarization_payload)
print(summary_response)

{'error': " `args[0]`: {'inputs': '', 'parameters': {'do_sample': False}} have the wrong format. The should be either of type `str` or type `list`", 'warnings': ["There was an inference error:  `args[0]`: {'inputs': '', 'parameters': {'do_sample': False}} have the wrong format. The should be either of type `str` or type `list`"]}
